## try/except/else语句

else分句则是提供没发生异常时要执行的处理程序

### try语句的工作方式

try语句启动时，python会标识当前的程序上下文，如果有异常发生，才能返回到这里。接下来发生什么，取决于try代码块语句执行时是否引发异常，以及这些异常的类型是否能和try监听的类型相匹配。

- 如果try代码块语句执行时发生了一个异常，python就会跳回try，把引发的异常对象赋值给as分句后的变量名，然后执行第一个复合引发异常的except分句下面的语句。当except在代码块执行后，控制权就会移交到整个try语句的后面继续执行。
- 如果异常发生在try语句块内，但却不能跟当前try语句的任何except分句描述的名字匹配，那么异常就会向上传递到程序之前最后一次进入的能够匹配的try中，如果找不到则找到默认异常处理
- 如果try首行底下执行的语句没有发生异常，python就会执行else行下的语句，之后控制权会从整个try语句的后面继续。

### try语句分句

|分句形式|说明|
| --- | --- |
| except: | 捕捉所有异常类型 |
| except name: | 只捕捉指定的异常 |
| except name as value: | 捕捉所列的异常并将该异常实例赋值给名称value | 
| except (name1, name2): | 捕捉任何列出的异常 |
| except (name1, name2) as value | 捕捉任何列出的异常，并将异常实例元组赋值给名称value |
| else： | 如果没有引发异常，就会运行 |
| finally: | 总是在退出try语句时运行此代码块 |

### 捕获任何(any)和所有(all)异常

- 没有列出异常名称的except分句，会捕获所有没在try语句靠前的except分句中列出的异常
- 在圆括号中列出的一组异常的except分句，会捕获任何被列出的异常

In [1]:
try:
    x = 'szq'
    x[5]
except:
    print('hello world')

hello world


虽然可以捕获所有异常，但是也可能捕获到与代码无关，意料之外的代码异常，如系统退出调用。

In [3]:
try:
    x = 'szq'
    x[5]
except Exception:
    # 通过Exception，忽略与系统退出相关的异常
    print('hello world')

hello world


## try/finally语句

如果在try中包含了finally分句，Python一定会在离开try语句时执行其语句代码块，无论try代码块在执行时是否发生了异常。

In [1]:
class MyError(Exception): pass 

In [2]:
def stuff(file):
    raise MyError()

In [3]:
file = open('data', 'w')
try:
    stuff(file)
finally:
    file.close()
print('not reached') # 上面发生错误，该句永远不会执行，但是finally中的会执行

MyError: 

### 合并try/except/finally语句

在python2.5之后，finally可以和try，except，else混合使用，可以在同一个try语句中混合使用。try语句必须至少有一个except或一个finally，其中else和finally是可选的，而且可能会有0个或多个except；但是如果出现一个else的话，必须有至少一个except。

## raise语句

如果要显式地触发异常，可以使用raise语句。一条raise语句的组成包括raise关键字，后面跟着可选的要引发的异常类或异常类的一个实例。
```python
raise instance
raise class
raise 
```
现在python的异常总是类的实例，实例要么是在raise之前创建的，要么就是raise语句中创建的。如果传入一个类，则python会对这个类调用不带参数的默认构造函数，以及来创建被引发的一个异常实例。

In [5]:
raise IndexError # 传入类则是隐式地创建实例
raise IndexError() # 实例
exc = IndexError()
raise exc # 提前创建实例

IndexError: 

In [11]:
try:
    raise IndexError()
except IndexError as X:
    print(type(X))

<class 'IndexError'>


### 作用域和try except变量

python3中会将异常引用名局限在对应的except块中，在块执行结束后该变量将不能再被使用。

In [18]:
try:
    1/0
except Exception as X:
    print(X)
    tempx = X
    raise # 重新引发当前错误

division by zero


ZeroDivisionError: division by zero

In [16]:
X # 出了作用域不能用了

NameError: name 'X' is not defined

In [17]:
tempx # 这样就能取出来了

ZeroDivisionError('division by zero')

### 异常链： raise from

异常有时能作为其他异常的响应而被触发：既可以是有意地被触发，也可以是由于其他新的程序错误而被触发

In [23]:
try:
    try:
        raise IndexError()
    except Exception as E:
        raise TypeError() from E 
except Exception as E:
    print(type(E))
    raise # 会把所有异常显示出来

<class 'TypeError'>


TypeError: 

## assert 语句

assert 可视为条件式地raise语句
```python
assert test, data

# 等价于
if __debug__:
    if not test:
        raise AssertionError(data)
```

如果test计算结果为假，那么Python就会引发异常，data项将作为异常构造函数地参数。 `__debug__`标志符是内置名称，除非有使用 -O标志

In [25]:
def f(x):
    assert x < 0, 'x must be negative'
    return x**2

In [26]:
f(1)

AssertionError: x must be negative

## with/as 上下文管理器

with/as 语句是常见try/finally用法模式的替代方案。with语句是基于一个对象协议，该协议用于指定在一段代码块前后运行的动作。　

```python
with expression as variable:
    code block
```

expression的计算结果是支持上下文协议的对象，而var则被赋值了其他要在语句中使用的对象。然后expression返回的对象可以在code-block开始前执行初始化程序，也可以在该code-block后执行终止化程序代码。有些内置的python对象已经得到强化，能够支持上下文管理协议，如文件打开，在code-block后会自动关闭文件。使用上下文管理起，可以使我们确保with代码块结束后，文档被关闭。

In [2]:
class TraceBlock:
    def message(self, args):
        print('running', args)
    def __enter__(self):
        print('starting with block')
        return self 
    def  __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is None:
            print('exited normally\n')
        else:
            print('raise an exception:' + str(exc_type))
            return False

In [ ]:
with TraceBlock() as action:
    